In [18]:
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

# Set any global values

In [19]:
torch.manual_seed(1)

# Load the training data

In [20]:
with open('training_data.pickle','rb') as f:
    training_data = pickle.load(f)

In [21]:
class_to_ix = {"Found":1, "Unfound":2}
all_letters = "()ab~&|>"

n_classes = len(class_to_ix)
n_letters = len(all_letters)

In [22]:
X = []
Y = []

for element in training_data:
    X.append(element[0])
    Y.append(element[1])

In [23]:
class TTPStyleDataset(Dataset):
    def __init__(self, X,Y):
        """
        Arguments:
            input(list): 
        """
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = [self.X[idx], self.Y[idx]]
        return sample

In [24]:
input_dataset = TTPStyleDataset(X,Y)

dataloader = DataLoader(input_dataset, shuffle=True, num_workers=0)

# Neural Network

In [25]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 64
rnn = RNN(n_letters, n_hidden, n_classes)

In [26]:
def classFromOutput(output):
    top_n, top_i = output.topk(1)
    class_i = top_i[0].item()
    return all_classes[class_i], class_i

In [27]:
num_epochs = 100
optimizer = torch.optim.ASGD(rnn.parameters(), lr=0.05)
criterion = nn.NLLLoss()
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        inputs = data[0]
        labels = data[1]
        hidden = rnn.initHidden()
        rnn.zero_grad()

        l = inputs.size()[1]

        for i in range(l):
            output, hidden = rnn(inputs[1], hidden)
        
        loss = criterion(output, labels)
        loss.backward()
        
        for p in rnn.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)

AttributeError: 'tuple' object has no attribute 'size'